<a href="https://colab.research.google.com/github/Naraujo13/bayes_spam/blob/master/bayes_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nícolas Oreques de Araujo

#Relatório

##Dataset:
A implementação abaixo de bayes utiliza o dataset de mensagens de spam de sms fornecito no curso 'Udacity's Machine Learning Engineer Nanodegree'.
 Este dataset é composto de 5572 mensagens de sms classificadas como spam ('spam') ou não spam ('ham').
 

## Problema:
O problema solucionado é referente a classificação textual. Neste caso em específico a classificação e detecção de spam em mensagens SMS.

##Pré-processamento:

Não foi realizado nenhum método de pré-processamento como stemming ou lemmatização.
Os labels foram trocados, onde spam tornou-se 1 e ham tornou-se 0, como forma de possibilitar a classificação.
Além disso, as mensagens foram convertidas para bag of words antes de serem usadas pelo modelo Bayes.

##Avaliação:

Para avaliação optou-se por cross-validaiton com holdout, onde o dataset foi previamente dividido em treino (75%) e teste (25%).

##Resultados:

Os resultados obtidos foram bastante satisfatórios, comprovando a aptidão do método de bayes para classificação textual.
Foram obtidos os seguintes resultados:

**Accuracy**: 0.9885139985642498

**Precision**: 0.9720670391061452

**Recall**: 0.9405405405405406

**F1**: 0.9560439560439562

#Pre-Processing:


In [0]:
import pandas as pd

In [0]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
input_file_path = '/content/drive/My Drive/Colab Notebooks/bayes_spam/spam.csv'

In [0]:
df = pd.read_table(input_file_path, names=['label', 'sms_message'])
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
#Convert labels
df.loc[:,'label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
df.head()

(5572, 2)


,label,sms_message
0,NaN,"Go until jurong point, crazy.. Available only ..."
1,NaN,Ok lar... Joking wif u oni...
2,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,NaN,U dun say so early hor... U c already then say...
4,NaN,"Nah I don't think he goes to usf, he lives aro..."


#Bag of Words

In [0]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [0]:
# Get words string representation
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [0]:
# Tokenize array
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

In [0]:
#convert to panda dataframe
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


#Separate datasets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['sms_message'], df['label'], random_state=1)
print('Total: {}'.format(df.shape[0]))
print('Train: {}'.format(X_train.shape[0]))
print('Test: {}'.format(X_test.shape[0]))

Total: 5572
Train: 4179
Test: 1393


#Apply bag of words to dataset

In [0]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

#Bayes

In [0]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
predictions = naive_bayes.predict(testing_data)

# Evaluate the model

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [0]:
print('Accuracy: {}'.format(accuracy_score(Y_test, predictions)))
print('Precision: {}'.format(precision_score(Y_test, predictions)))
print('Recall: {}'.format(recall_score(Y_test, predictions)))
print('F1: {}'.format(f1_score(Y_test, predictions)))

Accuracy: 0.9885139985642498
Precision: 0.9720670391061452
Recall: 0.9405405405405406
F1: 0.9560439560439562
